In [ ]:
# Import libraries and packages for the project 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import csv

print('- Finish importing packages')

In [ ]:
# Task 1: Login to Linkedin

# Task 1.1: Open Chrome and Access Linkedin login site
browser = webdriver.Chrome()
browser.get('https://www.linkedin.com/uas/login')
file = open('credentials.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]
elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)

elementID.submit()
print('- Finish Task 1: Login to Linkedin')

In [ ]:
# Task 2: Search for the profile we want to crawl

# Task 2.1: Locate the search bar element
search_field = browser.find_element_by_xpath('//*[@id="global-nav-typeahead"]/input')

# Task 2.2: Input the search query to the search bar
search_query = input('What profile do you want to scrape? ')
search_field.send_keys(search_query)

# Task 2.3: Search
search_field.send_keys(Keys.RETURN)

print('- Finish Task 2: Search for profiles')

In [ ]:
# Task 3: Scrape the URLs of the profiles

# Task 3.1: Write a function to extract the URLs of one page
def GetURL():
    page_source = BeautifulSoup(browser.page_source)
    profiles = page_source.find_all('a', class_ = 'app-aware-link')
    all_profile_URL = []
    for profile in profiles:
        profile_ID = profile.get('href')
        profile_URL = profile_ID
        if profile_URL not in all_profile_URL:
            if profile_URL.find("/in/") > 0: #Quitar URLs que no necesitamos
                all_profile_URL.append(profile_URL)     
    return all_profile_URL

# Task 3.2: Navigate through many page, and extract the profile URLs of each page

input_page = int(input('How many pages you want to scrape: '))
URLs_all_page = []
for page in range(input_page):
    URLs_one_page = GetURL()
    sleep(2)
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
    sleep(3)
    next_button = browser.find_element_by_class_name("artdeco-pagination__button--next")
    browser.execute_script("arguments[0].click();", next_button)
    URLs_all_page = URLs_all_page + URLs_one_page
    sleep(2)

print('- Finish Task 3: Scrape the URLs')

In [ ]:
# Task 4: Scrape the data of 1 Linkedin profile, and write the data to a .CSV file
headers = ['Name', 'Location', 'Job Title', 'URL']
info = []
link_excel = "borrar esta línea"

for linkedin_URL in URLs_all_page:
    page_source = BeautifulSoup(browser.page_source, "html.parser")
    soup = page_source
    try:
        sleep(2)
        browser.get(linkedin_URL)
        sleep(1)
        print('- Accessing profile: ', linkedin_URL)
        print(link_excel)
        link = link_excel
        info_div = page_source.find('div',{'class':'flex-1 mr5'})
        location_tag = soup.find('div', {'class': 'pb2 pv-text-details__left-panel'})
        info_loc = location_tag.find_all("span", {'class': 'text-body-small'})
        name_div = soup.find('div', {'class': 'pv-text-details__left-panel'})
        name =  name_div.find('h1').get_text().strip() #Remove unnecessary characters 
        print('--- Profile name is: ', name)
        location = info_loc[0].get_text().strip() #Remove unnecessary characters 
        print('--- Profile location is: ', location)
        info_title = soup.find('div', {'class': 'text-body-medium break-words'})
        title = info_title.get_text().strip()
        print('--- Profile title is: ', title)
        link_excel = browser.current_url
    except:
        print("Error al acceder")
        continue    
    print(link_excel)
    info.append([name, location, title, link])
    
    #writer.writerow({headers[0]:name, headers[1]:location, headers[2]:title, headers[3]:linkedin_URL})
    print('\n')
df = pd.DataFrame(info, columns=headers)
df.to_excel('data.xlsx', index=False)
print('Mission Completed!')